## Data Wrangling

In [47]:
#load python packages
import os
import pandas as pd
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import numpy as np

In [48]:
os.getcwd()

'C:\\Shalin\\Springboard bootcamp\\projects\\Capstone project 2\\Capstone Two'

In [49]:
os.listdir()

['.ipynb_checkpoints',
 'data',
 'embeddings',
 'figures',
 'models',
 'Quora Insincere Capstone Data Wrangling.ipynb',
 'sample_submission.csv',
 'test.csv',
 'train.csv']

In [50]:
#path=""
#os.chdir(path)

### Data Collection 

#### Load train and test csv files

In [51]:
file_name_1 = 'C:\\Shalin\\Springboard bootcamp\\projects\\Capstone project 2\\Capstone Two\\train.csv'
file_name_2 = 'C:\\Shalin\\Springboard bootcamp\\projects\\Capstone project 2\\Capstone Two\\test.csv'

In [52]:
# Create train and test dataframes by reading the csv files
train = pd.read_csv(file_name_1).drop('target', axis=1)
test = pd.read_csv(file_name_2)

In [53]:
#Concat both train and test files for initial pre-processing
quora_df = pd.concat([train, test])

In [54]:
quora_df.head(2)

,qid,question_text
0,00002165364db923c7e6,How did Quebec nationalists see their province...
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco..."


In [55]:
#Number of texts retrieved from merged dataframe
quora_df.shape[0]

1424381

### Data Organization

In [56]:
path = 'C:\\Shalin\\Springboard bootcamp\\projects\\Capstone project 2\\Capstone Two'

In [57]:
print ("The current working directory is %s" % path)

The current working directory is C:\Shalin\Springboard bootcamp\projects\Capstone project 2\Capstone Two


#### Create subfolders in your project directory

In [14]:
data = os.makedirs('data')

In [15]:
figures = os.makedirs('figures')

In [16]:
models = os.makedirs('models')

In [58]:
os.listdir()

['.ipynb_checkpoints',
 'data',
 'embeddings',
 'figures',
 'models',
 'Quora Insincere Capstone Data Wrangling.ipynb',
 'sample_submission.csv',
 'test.csv',
 'train.csv']

### Data Definition

In [59]:
quora_df.head(2)

,qid,question_text
0,00002165364db923c7e6,How did Quebec nationalists see their province...
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco..."


In [60]:
quora_df.tail(2)

,qid,question_text
375804,ffffb1f7f1a008620287,What are the causes of refraction of light?
375805,fffff85473f4699474b0,Climate change is a worrying topic. How much t...


In [61]:
quora_df.columns

Index(['qid', 'question_text'], dtype='object')

In [62]:
quora_df.dtypes

qid              object
question_text    object
dtype: object

In [63]:
quora_df.describe()

,qid,question_text
count,1424381,1424381
unique,1424381,1424379
top,c8ef7f0342b0d86b3678,#NAME?
freq,1,3


**Note**: 

- There are no numeric columns since the scope of this project is to perform text interpretation. Hence there are
no summary statistics that can be derived

- There are no null records since the count of qid and question_text is the same

**Convert the question_text column in the dataframe to list to list out all the different texts associated to records in the dataframe**

In [64]:
question_text_list = quora_df['question_text'].tolist()
question_text_list[0:10]

['How did Quebec nationalists see their province as a nation in the 1960s?',
 'Do you have an adopted dog, how would you encourage people to adopt and not shop?',
 'Why does velocity affect time? Does velocity affect space geometry?',
 'How did Otto von Guericke used the Magdeburg hemispheres?',
 'Can I convert montra helicon D to a mountain bike by just changing the tyres?',
 'Is Gaza slowly becoming Auschwitz, Dachau or Treblinka for Palestinians?',
 'Why does Quora automatically ban conservative opinions when reported, but does not do the same for liberal views?',
 'Is it crazy if I wash or wipe my groceries off? Germs are everywhere.',
 'Is there such a thing as dressing moderately, and if so, how is that different than dressing modestly?',
 'Is it just me or have you ever been in this phase wherein you became ignorant to the people you once loved, completely disregarding their feelings/lives so you get to have something go your way and feel temporarily at ease. How did things chan

**Find max and min length of question from the list**

In [65]:
max_len = 0
for length in question_text_list: 
    if len(length) > max_len: 
        max_len = len(length) 
        question = length 
print("Maximum length string is : " + question) 

Maximum length string is : What is [math]\frac{\int_{1x^5}^{3x^{-5}} \tan(\tan({\boxed{\int_{1x^0}^{1x^2} \sum_{\varpi=1}^{\infty} \int_{2x^{-3}}^{2x^2} \sum_{\alpha=7}^{\infty} \underbrace{\sqrt[2]{1x^5}}_{\text{Gauss's Law of Theoretical Probability.}} d\tau dx}}^{1x^0})) d\mu}{\int_{2x^{-3}}^{1x^5} \cos(\int_{2x^2}^{1x^{-3}} \frac{\sqrt[2]{\overbrace{\underbrace{\frac{3x^3+3x^5}{\sqrt[3]{2x^{-3}}}}_{\text{Gauss's Law of Theoretical Probability.}} \times \overbrace{\tan(2x^0)}^{\text{Gauss's Law of Theoretical Probability.}}-\sum_{4=7}^{\infty} \boxed{3x^{-5}}}^{\text{Inverse Function.}}}}{{\boxed{\int_{2x^2}^{2x^4} 3x^1 d9} \div \sum_{6=6}^{\infty} \sqrt[3]{2x^2}+\sqrt[4]{\sin(2x^0+3x^0)}}^{2x^{-4}}+\boxed{\frac{\vec{\boxed{\sum_{\gamma=10}^{\infty} 1x^{-5}}}}{\frac{\sum_{\iota=2}^{\infty} 1x^{-5}-\frac{3x^{-1}}{1x^{-4}}}{\sin(\tan(3x^{-2}))}}}} \times \boxed{\sqrt[2]{{{{\sqrt[5]{2x^5}}^{2x^{-1}}}^{2x^{-1}} \div \sum_{\chi=6}^{\infty} \int_{1x^4}^{2x^{-4}} 3x^2 d\vartheta+{2x^{-3}}^

**Alternative way of finding the max length of question from the list**

In [66]:
question_max = max(question_text_list, key = len) 
question_max

"What is [math]\\frac{\\int_{1x^5}^{3x^{-5}} \\tan(\\tan({\\boxed{\\int_{1x^0}^{1x^2} \\sum_{\\varpi=1}^{\\infty} \\int_{2x^{-3}}^{2x^2} \\sum_{\\alpha=7}^{\\infty} \\underbrace{\\sqrt[2]{1x^5}}_{\\text{Gauss's Law of Theoretical Probability.}} d\\tau dx}}^{1x^0})) d\\mu}{\\int_{2x^{-3}}^{1x^5} \\cos(\\int_{2x^2}^{1x^{-3}} \\frac{\\sqrt[2]{\\overbrace{\\underbrace{\\frac{3x^3+3x^5}{\\sqrt[3]{2x^{-3}}}}_{\\text{Gauss's Law of Theoretical Probability.}} \\times \\overbrace{\\tan(2x^0)}^{\\text{Gauss's Law of Theoretical Probability.}}-\\sum_{4=7}^{\\infty} \\boxed{3x^{-5}}}^{\\text{Inverse Function.}}}}{{\\boxed{\\int_{2x^2}^{2x^4} 3x^1 d9} \\div \\sum_{6=6}^{\\infty} \\sqrt[3]{2x^2}+\\sqrt[4]{\\sin(2x^0+3x^0)}}^{2x^{-4}}+\\boxed{\\frac{\\vec{\\boxed{\\sum_{\\gamma=10}^{\\infty} 1x^{-5}}}}{\\frac{\\sum_{\\iota=2}^{\\infty} 1x^{-5}-\\frac{3x^{-1}}{1x^{-4}}}{\\sin(\\tan(3x^{-2}))}}}} \\times \\boxed{\\sqrt[2]{{{{\\sqrt[5]{2x^5}}^{2x^{-1}}}^{2x^{-1}} \\div \\sum_{\\chi=6}^{\\infty} \\int_{1

In [67]:
question_min = min(question_text_list, key = len) 
question_min

'"'

### Data Cleaning

As identified in earlier step during data definition step, there are no null values that need to be cleaned up for the scope of this project

**Convert text to lowercase**

In [68]:
question_text_list = [v.lower() for v in question_text_list]
question_text_list[0:10]

['how did quebec nationalists see their province as a nation in the 1960s?',
 'do you have an adopted dog, how would you encourage people to adopt and not shop?',
 'why does velocity affect time? does velocity affect space geometry?',
 'how did otto von guericke used the magdeburg hemispheres?',
 'can i convert montra helicon d to a mountain bike by just changing the tyres?',
 'is gaza slowly becoming auschwitz, dachau or treblinka for palestinians?',
 'why does quora automatically ban conservative opinions when reported, but does not do the same for liberal views?',
 'is it crazy if i wash or wipe my groceries off? germs are everywhere.',
 'is there such a thing as dressing moderately, and if so, how is that different than dressing modestly?',
 'is it just me or have you ever been in this phase wherein you became ignorant to the people you once loved, completely disregarding their feelings/lives so you get to have something go your way and feel temporarily at ease. how did things chan

**Remove punctuation**
The following code removes this set of symbols [!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]:

In [69]:
question_text_list = [''.join(c for c in s if c not in string.punctuation) for s in question_text_list]
question_text_list[0:10]

['how did quebec nationalists see their province as a nation in the 1960s',
 'do you have an adopted dog how would you encourage people to adopt and not shop',
 'why does velocity affect time does velocity affect space geometry',
 'how did otto von guericke used the magdeburg hemispheres',
 'can i convert montra helicon d to a mountain bike by just changing the tyres',
 'is gaza slowly becoming auschwitz dachau or treblinka for palestinians',
 'why does quora automatically ban conservative opinions when reported but does not do the same for liberal views',
 'is it crazy if i wash or wipe my groceries off germs are everywhere',
 'is there such a thing as dressing moderately and if so how is that different than dressing modestly',
 'is it just me or have you ever been in this phase wherein you became ignorant to the people you once loved completely disregarding their feelingslives so you get to have something go your way and feel temporarily at ease how did things change']

In [70]:
question_max = max(question_text_list, key = len) 
question_max

'in star trek 2013 why did they \n\nspoilers\nspoilers\nspoilers\nspoilers\n\n1make warping look quite a bit like an hyperspace jump\n2what in the world were those bright particles as soon as they jumped\n3why in the world did they make it possible for two entities to react in warp space in separate jumps\n4why did spock get emotions for this movie\n5what was the point of hiding the enterprise underwater\n6when they were intercepted by the dark ship how come they reached earth when they were far away from heri dont seem to remember the scene where they warp to earth\n7how did the ship enter earths atmosphere when it wasnt even in orbit\n8when scotty opened the door of the black ship  how come pike and khan didnt slow down'

In [71]:
question_min = min(question_text_list, key = len) 
question_min

''

As you can see above, there are whitespaces, new line characters which can be removed (on deriving length and min length strings)

**Remove newline characters and whitespaces**

In [72]:
question_text_list[0:10]

['how did quebec nationalists see their province as a nation in the 1960s',
 'do you have an adopted dog how would you encourage people to adopt and not shop',
 'why does velocity affect time does velocity affect space geometry',
 'how did otto von guericke used the magdeburg hemispheres',
 'can i convert montra helicon d to a mountain bike by just changing the tyres',
 'is gaza slowly becoming auschwitz dachau or treblinka for palestinians',
 'why does quora automatically ban conservative opinions when reported but does not do the same for liberal views',
 'is it crazy if i wash or wipe my groceries off germs are everywhere',
 'is there such a thing as dressing moderately and if so how is that different than dressing modestly',
 'is it just me or have you ever been in this phase wherein you became ignorant to the people you once loved completely disregarding their feelingslives so you get to have something go your way and feel temporarily at ease how did things change']

- Remove newline characters

In [73]:
question_text_list = [re.sub(r'\n\d', '?', text).replace("\nspoilers","").replace("\n","") for text in question_text_list]
question_text_list

['how did quebec nationalists see their province as a nation in the 1960s',
 'do you have an adopted dog how would you encourage people to adopt and not shop',
 'why does velocity affect time does velocity affect space geometry',
 'how did otto von guericke used the magdeburg hemispheres',
 'can i convert montra helicon d to a mountain bike by just changing the tyres',
 'is gaza slowly becoming auschwitz dachau or treblinka for palestinians',
 'why does quora automatically ban conservative opinions when reported but does not do the same for liberal views',
 'is it crazy if i wash or wipe my groceries off germs are everywhere',
 'is there such a thing as dressing moderately and if so how is that different than dressing modestly',
 'is it just me or have you ever been in this phase wherein you became ignorant to the people you once loved completely disregarding their feelingslives so you get to have something go your way and feel temporarily at ease how did things change',
 'what can you

In [74]:
#question_text_list = [re.sub(r'\n[0-9]?(spoilers)?', '?', x).replace('???????','') for x in question_text_list]
#question_text_list 

- Remove whitespaces

In [75]:
question_text_list = [x.strip() for x in question_text_list]
question_text_list 

['how did quebec nationalists see their province as a nation in the 1960s',
 'do you have an adopted dog how would you encourage people to adopt and not shop',
 'why does velocity affect time does velocity affect space geometry',
 'how did otto von guericke used the magdeburg hemispheres',
 'can i convert montra helicon d to a mountain bike by just changing the tyres',
 'is gaza slowly becoming auschwitz dachau or treblinka for palestinians',
 'why does quora automatically ban conservative opinions when reported but does not do the same for liberal views',
 'is it crazy if i wash or wipe my groceries off germs are everywhere',
 'is there such a thing as dressing moderately and if so how is that different than dressing modestly',
 'is it just me or have you ever been in this phase wherein you became ignorant to the people you once loved completely disregarding their feelingslives so you get to have something go your way and feel temporarily at ease how did things change',
 'what can you

In [76]:
question_max = max(question_text_list, key = len) 
question_max

'in star trek 2013 why did they ?make warping look quite a bit like an hyperspace jump?what in the world were those bright particles as soon as they jumped?why in the world did they make it possible for two entities to react in warp space in separate jumps?why did spock get emotions for this movie?what was the point of hiding the enterprise underwater?when they were intercepted by the dark ship how come they reached earth when they were far away from heri dont seem to remember the scene where they warp to earth?how did the ship enter earths atmosphere when it wasnt even in orbit?when scotty opened the door of the black ship  how come pike and khan didnt slow down'

In [77]:
question_min = min(question_text_list, key = len) 
question_min

''

In [78]:
quora_df.question_text.count()

1424381

In [79]:
len(question_text_list)

1424381

#### Handling missing values

In [80]:
# Find out what the missing element in the list contains
null_list = [text for text in question_text_list if len(text) < 1]
null_list

['']

In [81]:
question_text_list.remove('')

In [82]:
len(question_text_list)

1424380

**Remove stop words**
- “Stop words” are the most common words in a language like “the”, “a”, “on”, “is”, “all”. These words do not carry important meaning and are usually removed from texts. It is possible to remove stop words using Natural Language Toolkit (NLTK), a suite of libraries and programs for symbolic and statistical natural language processing.

In [83]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gosal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gosal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
stop_words = set(stopwords.words('english'))

quora_list = question_text_list[:] #make a copy of the list
for word in question_text_list: # iterate over list
    if word in stop_words: 
        quora_list.remove(word) # remove word from quora_list if it is a stopword

In [ ]:
quora_list[0:10]